In [19]:
import requests
import json
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

In [20]:
from datetime import timedelta, date

dates = []

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2021, 11, 1) #Début: 2020, 7, 1
end_dt = date(2021, 12, 15)
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))
#print(dates)

In [24]:
match = pd.DataFrame(columns = ['id', 'date', 'time', 'year_compet', 'id_compet', 'name_compet', 'jour_compet', 'local_id', 'visitor_id', 'local_team', 'visitor_team', 'teams', 'resultat', 'home_goal', 'away_goal', 'vainqueur'])
#leagues_id = [25]
leagues_id = [107,117,204,19,20,8,10,16,92,9,27,93,7,89,2,1,37,25,69,11,22,17,18,205,128,95,24,58,54,346,600,22,29,46,12,13,68,706,2149,673,674,675,119,120,121,122,124,59,14,17,36,50,31,1780,2492,30,2006,97]
#leagues_id = [107,117,10,1,8,7,16,19,69,68,24,675,9,27,12,37,36,11,17,22,18,13,14,58,25,2,93,89,92] #FTE LEAGUE: + 1666-USL_USA, 21-PSL, 87-LEAGUE_ONE, 88-LEAGUE_TWO

id_mts = []
dates = ['2022-01-02']
k = 0
for date in dates:
    url_compet = "https://fast.besoccer.com/scripts/api/api.php?&key=b3fcd6725e03f4e5d588f6624cac5522&format=json&site=ResultadosAndroid&req=home_matches&locale=24&date="+date+"&extra=120&lang=fr&vr=6"
    reponse = requests.get(url_compet).json()
    competitions = reponse.get('competitions')
    #print(reponse)
    
    '''for competition in competitions:
        id_competition = competition.get('id')
        print(competition.get('id'), '-', competition.get('n'))'''
    
    for competition in competitions:
        id_competition = competition.get('id')
        #print(competition.get('id'), '-', competition.get('n'))
        year_compet = competition.get('y')
        name_compet = competition.get('n')
        jour_compet = competition.get('t')
        
        if int(id_competition) in leagues_id:
            mts = competition.get('mt')
            
            for mt in mts:
                if mt.get('s') == 1 or mt.get('s') == -1 or mt.get('s') == 0: #Si le match n'est pas reporté
                    #print(mt)
                    time = mt.get('ld')
                    id_mt = mt.get('id')
                    id_mts.append(id_mt)
                    local_id = mt.get('lid')
                    visitor_id = mt.get('vid')
                    local_team = mt.get('l')
                    visitor_team = mt.get('v')
                    
                    if mt.get('w') == 1:
                        vainqueur = 0
                    elif mt.get('w') == 2:
                        vainqueur = 2
                    elif mt.get('w') == None:
                        vainqueur = 1
                    
                    if mt.get('r'):
                        home_goal = mt.get('r').split('-')[0]
                        away_goal = mt.get('r').split('-')[1]
                        if mt.get('hp'):
                            home_goal = int(home_goal.split('(')[0])
                            away_goal = int(away_goal.split(')')[1])
                        else:
                            home_goal = int(home_goal)
                            away_goal = int(away_goal)
                            
                        resultat = str(home_goal) + '-' + str(away_goal)
                        
                    typeItems(year_compet, id_mt)
                    
                    last_xGs_local = avg_xG(local_id)
                    last_xGs_visitor = avg_xG(visitor_id)
                    #print(last_xGs_local[4])
                    
                    if last_xGs_local and last_xGs_visitor:
                        if last_xGs_local[4] > 10 and last_xGs_visitor[4] > 10:
                            
                            match.loc[k, 'l_xG'] = float(last_xGs_local[3])
                            match.loc[k, 'v_xG'] = float(last_xGs_visitor[3])

                            match.loc[k, 'l_xG10'] = float(last_xGs_local[5])
                            match.loc[k, 'v_xG10'] = float(last_xGs_visitor[5])
                            
                            match.loc[k, 'l_xG9'] = float(last_xGs_local[10])
                            match.loc[k, 'v_xG9'] = float(last_xGs_visitor[10])
                            
                            match.loc[k, 'l_xG8'] = float(last_xGs_local[9])
                            match.loc[k, 'v_xG8'] = float(last_xGs_visitor[9])

                            match.loc[k, 'l_xG7'] = float(last_xGs_local[2])
                            match.loc[k, 'v_xG7'] = float(last_xGs_visitor[2])
                            
                            match.loc[k, 'l_xG6'] = float(last_xGs_local[8])
                            match.loc[k, 'v_xG6'] = float(last_xGs_visitor[8])

                            match.loc[k, 'l_xG5'] = float(last_xGs_local[1])
                            match.loc[k, 'v_xG5'] = float(last_xGs_visitor[1])
                            
                            match.loc[k, 'l_xG4'] = float(last_xGs_local[6])
                            match.loc[k, 'v_xG4'] = float(last_xGs_visitor[6])

                            match.loc[k, 'l_xG3'] = float(last_xGs_local[0])
                            match.loc[k, 'v_xG3'] = float(last_xGs_visitor[0])
                            
                            match.loc[k, 'l_xG2'] = float(last_xGs_local[7])
                            match.loc[k, 'v_xG2'] = float(last_xGs_visitor[7])
                            
                            l_avg_xG2_10 = round((last_xGs_local[7] + last_xGs_local[0] + last_xGs_local[6] + last_xGs_local[1] + last_xGs_local[8] + last_xGs_local[2]) / 6, 2)
                            v_avg_xG2_10 = round((last_xGs_visitor[7] + last_xGs_visitor[0] + last_xGs_visitor[6] + last_xGs_visitor[1] + last_xGs_visitor[8] + last_xGs_visitor[2]) / 6, 2)

                            match.loc[k, 'l_avg_xG2_10'] = float(l_avg_xG2_10)
                            match.loc[k, 'v_avg_xG2_10'] = float(v_avg_xG2_10)
                            
                            match.loc[k, 'l_nbM'] = int(last_xGs_local[4])
                            match.loc[k, 'v_nbM'] = int(last_xGs_visitor[4])

                            '''futur_xGs_local = match.loc[k, 'local_xG']
                            futur_xGs_visitor = match.loc[k, 'visitor_xG']

                            last_res = poisson_percent(last_xGs_local[3], last_xGs_visitor[3], 30)[0]
                            futur_res = poisson_percent(futur_xGs_local, futur_xGs_visitor, 30)[0]

                            last_1 = percent_to_odd(last_res[0])
                            last_N = percent_to_odd(last_res[1])
                            last_2 = percent_to_odd(last_res[2])

                            futur_1 = percent_to_odd(futur_res[0])
                            futur_N = percent_to_odd(futur_res[1])
                            futur_2 = percent_to_odd(futur_res[2])

                            drop_1 = dropping_me(last_1, futur_1)
                            drop_N = dropping_me(last_N, futur_N)
                            drop_2 = dropping_me(last_2, futur_2)

                            match.loc[k, 'drop_1'] = drop_1
                            match.loc[k, 'drop_N'] = drop_N
                            match.loc[k, 'drop_2'] = drop_2  '''                      
                        

                    match.loc[k, 'id'] = id_mt
                    match.loc[k, 'date'] = date
                    match.loc[k, 'time'] = time
                    match.loc[k, 'year_compet'] = year_compet
                    match.loc[k, 'id_compet'] = id_competition
                    match.loc[k, 'name_compet'] = name_compet
                    match.loc[k, 'jour_compet'] = jour_compet
                    match.loc[k, 'local_id'] = local_id
                    match.loc[k, 'visitor_id'] = visitor_id
                    match.loc[k, 'local_team'] = local_team
                    match.loc[k, 'visitor_team'] = visitor_team
                    match.loc[k, 'teams'] = local_team + " - " + visitor_team
                    if mt.get('r'):
                        match.loc[k, 'resultat'] = resultat
                        match.loc[k, 'home_goal'] = home_goal
                        match.loc[k, 'away_goal'] = away_goal
                        match.loc[k, 'vainqueur'] = vainqueur
                k = k +1
                #print(k)
print('FINISH !')
match.head()

FINISH !


,id,date,time,year_compet,id_compet,name_compet,jour_compet,local_id,visitor_id,local_team,...,l_xG4,v_xG4,l_xG3,v_xG3,l_xG2,v_xG2,l_avg_xG2_10,v_avg_xG2_10,l_nbM,v_nbM
0,20962,2022-01-02,15:00,2022,1,Liga Santander,Journée 19,1217,2107,Getafe,...,0.33,6.00,0.33,4.00,0.20,2.00,0.74,5.75,20.0,25.0
1,20958,2022-01-02,17:15,2022,1,Liga Santander,Journée 19,369,2080,Atlético,...,0.71,2.00,0.33,1.67,0.50,5.00,0.69,2.31,25.0,24.0
2,20969,2022-01-02,17:15,2022,1,Liga Santander,Journée 19,975,4235,Elche,...,1.17,1.75,0.80,2.00,1.00,3.00,1.15,2.39,20.0,20.0
3,20957,2022-01-02,19:30,2022,1,Liga Santander,Journée 19,137,2120,Alavés,...,0.40,1.00,0.33,0.57,0.29,1.33,0.46,1.00,20.0,26.0
4,20960,2022-01-02,19:30,2022,1,Liga Santander,Journée 19,486,712,Real Betis,...,1.50,1.50,2.00,2.50,1.20,2.50,1.78,2.36,26.0,20.0


In [25]:
detail = match[['vainqueur', 'local_team', 'local_elo', 'visitor_team', 'visitor_elo', 'resultat', 'l_avg_xG2_10', 'v_avg_xG2_10']]
detail

,vainqueur,local_team,local_elo,visitor_team,visitor_elo,resultat,l_avg_xG2_10,v_avg_xG2_10
0,NaN,Getafe,1643.0,Real Madrid,3786.0,NaN,0.74,5.75
1,NaN,Atlético,3728.0,Rayo Vallecano,1685.0,NaN,0.69,2.31
2,NaN,Elche,1426.0,Granada,1876.0,NaN,1.15,2.39
3,NaN,Alavés,1660.0,R. Sociedad,2471.0,NaN,0.46,1.00
4,NaN,Real Betis,2194.0,Celta,2552.0,NaN,1.78,2.36
5,NaN,Mallorca,1307.0,Barcelona,3768.0,NaN,1.87,0.96
6,NaN,Brentford,1055.0,Aston Villa,1578.0,NaN,0.29,0.98
7,NaN,Everton,2643.0,Brighton & Hove Albion,1382.0,NaN,0.52,1.53
8,NaN,Leeds United,1287.0,Burnley,1851.0,NaN,0.29,0.49
10,NaN,Chelsea,3432.0,Liverpool,3229.0,NaN,2.33,1.15


In [26]:
match.sort_values(by = 'time').reset_index().to_csv('W:/Documents/GitHub/soccer_events/models/BESOCCER_MyxG.csv', index=False)

In [3]:
def avg_xG(rid_dteam):             
    url = "https://apiclient.besoccerapps.com/scripts/api/api.php?key=b3fcd6725e03f4e5d588f6624cac5522&tz=Europe/Madrid&format=json&req=matches_team&id="+str(rid_dteam)

    response = requests.request("GET", url)

    #print(response.text)
    competitions = response.json().get('matches')
    matches_by_team = pd.DataFrame(columns = ['datetime', 'year', 'matche_id', 'league_id', 'competition_name', 'team_id', 'team_name', 'goals_for', 'goals_against'])
    k = 0
    for competition in competitions:
        competition_id = competition.get('id')
        league_id = competition.get('league_id')
        competition_name = competition.get('name')
        year = competition.get('year')
        matches = competition.get('matches') #OBJET
        #print(competition_name, '-', len(matches))
        if matches != None and int(year) > 2020 and int(league_id) != 62807:
            for matche in matches:
                status = matche.get('status')
                if status == 1:
                    #print(matche)
                    matche_id = matche.get('id')
                    t1_id = int(matche.get('t1_id'))
                    t2_id = int(matche.get('t2_id'))
                    
                    t1_name = matche.get('t1_name')
                    t2_name = matche.get('t2_name')
                    shedule = matche.get('shedule')
                    r1 = int(matche.get('r1'))
                    r2 = int(matche.get('r2'))
                    
                    if datetime.strptime(date, '%Y-%m-%d') > datetime.strptime(shedule, '%Y-%m-%d %H:%M:%S'):
                        if t1_id == int(rid_dteam):
                            matches_by_team.loc[k, 'datetime'] = shedule
                            matches_by_team.loc[k, 'year'] = year
                            matches_by_team.loc[k, 'matche_id'] = matche_id
                            matches_by_team.loc[k, 'league_id'] = league_id
                            matches_by_team.loc[k, 'competition_name'] = competition_name
                            matches_by_team.loc[k, 'team_id'] = t1_id
                            matches_by_team.loc[k, 'team_name'] = t1_name
                            matches_by_team.loc[k, 'goals_for'] = r1
                            matches_by_team.loc[k, 'goals_against'] = r2
                            if r1 > r2:
                                matches_by_team.loc[k, 'winner'] = 'w'
                            if r1 == r2:
                                matches_by_team.loc[k, 'winner'] = 'd'
                            if r1 < r2:
                                matches_by_team.loc[k, 'winner'] = 'l'


                        elif t2_id == int(rid_dteam):
                            matches_by_team.loc[k, 'datetime'] = shedule
                            matches_by_team.loc[k, 'year'] = year
                            matches_by_team.loc[k, 'matche_id'] = matche_id
                            matches_by_team.loc[k, 'league_id'] = league_id
                            matches_by_team.loc[k, 'competition_name'] = competition_name
                            matches_by_team.loc[k, 'team_id'] = t2_id
                            matches_by_team.loc[k, 'team_name'] = t2_name
                            matches_by_team.loc[k, 'goals_for'] = r2
                            matches_by_team.loc[k, 'goals_against'] = r1
                            if r1 < r2:
                                matches_by_team.loc[k, 'winner'] = 'w'
                            if r1 == r2:
                                matches_by_team.loc[k, 'winner'] = 'd'
                            if r1 > r2:
                                matches_by_team.loc[k, 'winner'] = 'l'

                        #print(shedule, '-', t1_id, ':', r1, 'vs', t2_id, ':', r2)
                        k = k +1
    #print(matches_by_team)                   
    nb_match = k
    matches_by_team = matches_by_team.sort_values(by = 'datetime').reset_index()
    #print(matches_by_team)
    count_matche = 1
    count_goals_for = 0
    count_goals_against = 0
        
    for k in range(len(matches_by_team)):
        goals_for = matches_by_team.loc[k, 'goals_for']
        goals_against = matches_by_team.loc[k, 'goals_against']
        count_goals_for = count_goals_for + goals_for
        count_goals_against = count_goals_against + goals_against
        offensive = count_goals_for / count_matche
        defensive = count_goals_against / count_matche
        matches_by_team.loc[k, 'offensive'] = offensive
        matches_by_team.loc[k, 'defensive'] = defensive
        if defensive > 0:
            expected_goals = offensive / defensive
            matches_by_team.loc[k, 'expected_goals'] = expected_goals
        else:
            matches_by_team.loc[k, 'expected_goals'] = offensive
    
        matches_by_team.loc[k, 'expected_goals_l2'] = xG_last_n_games(k, 2, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l3'] = xG_last_n_games(k, 3, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l4'] = xG_last_n_games(k, 4, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l5'] = xG_last_n_games(k, 5, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l6'] = xG_last_n_games(k, 6, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l7'] = xG_last_n_games(k, 7, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l8'] = xG_last_n_games(k, 8, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l9'] = xG_last_n_games(k, 9, matches_by_team)
        matches_by_team.loc[k, 'expected_goals_l10'] = xG_last_n_games(k, 10, matches_by_team)
        
        count_matche = count_matche + 1 
        
    if 'expected_goals_l2' in matches_by_team.columns:
        expected_goals_l2 = round(matches_by_team.loc[:,'expected_goals_l2'].iloc[-1:], 2)
        expected_goals_l3 = round(matches_by_team.loc[:,'expected_goals_l3'].iloc[-1:], 2)
        expected_goals_l4 = round(matches_by_team.loc[:,'expected_goals_l4'].iloc[-1:], 2)
        expected_goals_l5 = round(matches_by_team.loc[:,'expected_goals_l5'].iloc[-1:], 2)
        expected_goals_l6 = round(matches_by_team.loc[:,'expected_goals_l6'].iloc[-1:], 2)
        expected_goals_l7 = round(matches_by_team.loc[:,'expected_goals_l7'].iloc[-1:], 2)
        expected_goals_l8 = round(matches_by_team.loc[:,'expected_goals_l8'].iloc[-1:], 2)
        expected_goals_l9 = round(matches_by_team.loc[:,'expected_goals_l9'].iloc[-1:], 2)
        expected_goals_l10 = round(matches_by_team.loc[:,'expected_goals_l10'].iloc[-1:], 2)

        expected_goals = round(matches_by_team.loc[:,'expected_goals'].iloc[-1:], 2)
        
        res = [expected_goals_l3, expected_goals_l5, expected_goals_l7, expected_goals, nb_match, expected_goals_l10, expected_goals_l4, expected_goals_l2, expected_goals_l6, expected_goals_l8, expected_goals_l9]
    else:
        res = False

    #print(matches_by_team)

    return res


In [4]:
def xG_last_n_games(k, n, matches_by_team):
        #LAST GAMES N
        if k > n-2: 
            count_matche_l3 = 1
            count_goals_for_l3 = 0
            count_goals_against_l3 = 0
            matches_by_team_l3 = matches_by_team.loc[:k].iloc[-n:].reset_index()

            for k_l3 in range(len(matches_by_team_l3)):
                goals_for_l3 = matches_by_team_l3.loc[k_l3, 'goals_for']
                goals_against_l3 = matches_by_team_l3.loc[k_l3, 'goals_against']
                count_goals_for_l3 = count_goals_for_l3 + goals_for_l3
                count_goals_against_l3 = count_goals_against_l3 + goals_against_l3
                offensive_l3 = count_goals_for_l3 / count_matche_l3
                defensive_l3 = count_goals_against_l3 / count_matche_l3
                count_matche_l3 = count_matche_l3 + 1

            if defensive_l3 > 0:
                expected_goals_l3 = offensive_l3 / defensive_l3
            else:
                expected_goals_l3 = offensive_l3
            return expected_goals_l3

In [5]:
def typeItems(year, id_mt):
        
    url_mt = "https://fast.besoccer.com/scripts/api/api.php?&key=b3fcd6725e03f4e5d588f6624cac5522&format=json&site=ResultadosAndroid&req=match_elo&id="+id_mt+"&year="+year+"&t1=-1&t2=-1"
    reponse = requests.get(url_mt).json()
    analysis = reponse.get('analysis')

    for une_analysi in analysis:
        typeItem = une_analysi.get('typeItem')

        #print(une_analysi)
        if typeItem == 1:
            local = une_analysi.get('local_elo')
            visitor = une_analysi.get('visitor_elo')
            probabilities_elo = une_analysi.get('probabilities_elo')
            
            local_elo = local.get('elo_points_real')
            visitor_elo = visitor.get('elo_points_real')
            
            match.loc[k, 'local_elo'] = local_elo
            match.loc[k, 'visitor_elo'] = visitor_elo

        if typeItem == 2:
            local_win_prob = une_analysi.get('local_win_prob')
            draw_prob = une_analysi.get('draw_prob')
            visitor_win_prob = une_analysi.get('visitor_win_prob')

            local_expect_goals = local_win_prob.get('expect_goals')
            visitor_expect_goals = visitor_win_prob.get('expect_goals')
            
            match.loc[k, 'local_xG'] = local_expect_goals
            match.loc[k, 'visitor_xG'] = visitor_expect_goals


In [6]:
def percent_to_odd(percent):
    return float((round(1 / percent * 100, 2)))

def dropping_me(odd_arrive, odd_depart):
    return float(round((odd_arrive - odd_depart)/(odd_depart - 1) * 100, 2))

def fact(nbr):
    f = 1
    for i in range(1, nbr+1):
        f = f * i
    return f

def poisson_percent(lxG, vxG, max_goals):
    local_distrib = []
    visitor_distrib = []
    
    matrice_goals = {}
    
    local_lambda = lxG
    visitor_lambda = vxG
    
    for i in range(0, max_goals+1):
        k = i
        kfact = fact(i)
        
        local = math.exp(-local_lambda) * (math.pow(local_lambda, k) / kfact)
        visitor = math.exp(-visitor_lambda) * (math.pow(visitor_lambda, k) / kfact)
        
        local_distrib.append(local)
        visitor_distrib.append(visitor)
        
    home_prob = 0
    draw_prob = 0
    away_prob = 0
    
    for local_i in range(0, len(local_distrib)):
        for visitor_i in range(0, len(visitor_distrib)):
            
            local_prob_score = local_distrib[local_i]
            visitor_prob_score = visitor_distrib[visitor_i]
            
            if local_i > visitor_i:
                home_prob = local_prob_score * visitor_prob_score + home_prob
                
            if local_i == visitor_i:
                draw_prob = local_prob_score * visitor_prob_score + draw_prob
                
            if local_i < visitor_i:
                away_prob = local_prob_score * visitor_prob_score + away_prob
                
            if local_i <= 5 and visitor_i <= 5:
                score = str(local_i)+ '-'+str(visitor_i)
                matrice_goals[score] = local_prob_score * visitor_prob_score

    home_prob = float(round(home_prob * 100, 2))
    draw_prob = float(round(draw_prob * 100, 2))
    away_prob = float(round(away_prob * 100, 2))
    
    res = [home_prob, draw_prob, away_prob]
    
    return [res, matrice_goals]

In [7]:
dropping_me(1.5, percent_to_odd(poisson_percent(2.07, 0.86, 25)[0][0]))

-5.66